# DCR Graph Conformance Checking Tool

This is an implementation of a Conformance Checking tool with use of declarative language models. 

In [ ]:
#installing necassary modules/dependencies to the session

import sys
!{sys.executable} -m pip install -U xmltodict 

In [ ]:
import pandas as pd
import glob, os

import xmltodict
from collections import OrderedDict
from ordered_set import OrderedSet



## Trace extraction / Parser

Class for extracting all activities present in the trace log. 

### NB! Only operable on scenario/trace XML logs from dcrgraphs.net.  XML logs must be processed as dictionaries via the xmltodict module first in order to work properly

In [ ]:
class Trace:
    """
    A class used to handle log traces executed from an DCR model
    
    ...
    
    Attributes
    ----------
    _trace : OrderedDict
        a dictionary containing the trace log
    
    _events : list
        a list containing only the events from the trace log
    
    Methods
    -------
    extract_events()
        extracts all the events from the raw log in sequence
    
    """
    
    def __init__(self, trace):
        """
        Parameters
        ----------
        _trace : OrderedDict
            a dicitonary containing the trace log
        
        """
        
        self._trace = trace
        self._events = []

    
    # Extracting events
    def extract_events(self):
        """
        extracts all event containers from the parsed XML log and appends then into sequence.
        does not provide return value, method manipulates object attributes in-place
        
        """
        
        for log in self._trace['log']['trace']['event']:
            self._events.append(log)


## Process Model extractor / Parser 
 Class for extracting and handling data produced by the model.
### NB! Only operable on model log from dcrprahps.net. XML log must be processed as dicitonaries with xmltodict first in order to work properly

In [ ]:
class ProcessModel:
    """
    A class for creating a model object of a DCR model.
    
    ...
    
    Attributes 
    ----------
    _model : OrderedDict
        dictionary representing the DCR graph model
    
    _relations : dict
        dictionary containing combinations of activity relations as keys, together with corresponding 
        constraints as values. excludes all relations which are governed by data input
    
    _time : dict
        dictionary containing time-dependent relations, e.g constraints with delay or deadline control
        relation as key and temporal data as value
    
    _expressions : dict
        dictionary containing information of data related choices and input, together with corresponding 
        link/action ID. data guard as key, with correpsonding link/mapping ID as value
        
    _expression_relations : dict
        dictionary containing information of data-dependant constraints: link/mapping ID as key, 
        activity relation and affecting constraint as value
    
    _subevents : dict
        dictionary containing all event ID's of nested events inside a subprocess/nesting process as keys,
        with the corresponding subprocess/nesting ID as values
        
    _subprocess : dict
        dicitonary containing all subprocess/nesting ID's as keys with corresponding nested 
        subevents ID's as values
        
    _activities : dict
        dictionary containing all activities as keys and their current state (included, pending, executed) 
        as values
        
    _included : list
        list containing all activities which are included according to the model
        
    _pending = list
        list containing all activities which are pending according to the model
        
    _executed = list
        list containing all activities which are executed according to the model
        
    Methods
    -------
    extract_relations()
        extracts all the relations comprised by the constraints of the DCR model
        
    extract_nested_activities()
        extracts information on nested/subprocess types and activities 
        
    extract_activities()
        extracts all activity ID's and append tuple of state
        
    extract_expressions()
        extracts all data-dependent relations, data gaurds and their corresponding link/mapping ID
        
    extract_markings()
        extracts the initial state of the activities corresponding to the model
        
    extract_all()
        performs all above methods in sequence

    """
    
    def __init__(self, model):
        """
        Parameters
        ----------
        model : OrderedDict()
            a dictionary representing the DCR graph model
        """
        
        self._model = model
        self._relations = {}
        self._time = {}
        self._expressions = {}
        self._expression_relations = {}
        self._subevents = {}
        self._subprocess = {}
        self._activities = {}
        self._included = []
        self._pending = []
        self._executed = []

    # method for extracting all the relations between activities and the corresponding constraints
    def extract_relations(self):
        """
        method for extracting the relations between activities comprised by the constraints of the DCR model.
        does not provide return value, method manipulates object attributes in-place
        """
        
        # traverse the dictionary
        for events in self._model.get('dcrgraph').get('specification').get('constraints').values():    
            
            if events is not None: 
                
                # iterate through all constraints and relations
                # cast constraints to lists in order to handle multiple constraints per relation
                for constraint, relation in events.items(): 
                    if type(relation) != list: 
                            relation = [relation]
                            
                    for i in range(0, len(relation)):
                        
                        # fetch all data-dependent relations
                        if '@expressionId' in relation[i].keys():
                            if relation[i]['@expressionId'] not in self._expression_relations.items():
                                self._expression_relations[relation[i]['@expressionId']] = {
                                    (relation[i]['@sourceId'], relation[i]['@targetId']) : constraint} 
                        
                        # fetch all non-data dependent relations 
                        else:
                            if (relation[i]['@sourceId'], relation[i]['@targetId']) not in self._relations.keys():
                                self._relations[relation[i]['@sourceId'], 
                                                relation[i]['@targetId']] = [constraint]
                            else:
                                # append constraints to existing relations
                                self._relations[(relation[i]['@sourceId'], 
                                                 relation[i]['@targetId'])].append(constraint)

                        # fetch data on constraints with delays/deadlines
                        if relation[i]['@time'] != '':
                            self._time[(relation[i]['@sourceId'], 
                                        relation[i]['@targetId'])] = relation[i]['@time']
                        
    
    # method for fetching nested events and its subprocesses
    def extract_nested_activities(self):
        """
        method for extracting information on nested/subprocess types and activities.
        does not provide return value, method manipulates object attributes in-place
        """
        
        # traverse the dicitonary
        for events in self._model.get('dcrgraph').get('specification').get('resources').get('events').values():
            for subprocess in events:
                
                # init nesting/subprocesses type
                if '@type' in subprocess.keys():                    
                    if subprocess['@type'] not in self._subevents.keys():
                        self._subevents[subprocess['@type']] = {}
                        self._subprocess[subprocess['@type']] = {}
                        self._subprocess[subprocess['@type']][subprocess['@id']] = []
                    else:
                        self._subprocess[subprocess['@type']][subprocess['@id']] = []
                    
                    # iterate through all nested activities and fetch both activity ID together
                    # with nesting ID and nesting ID together with activity ID into dictionaries
                    for subevent in subprocess.get('event'):
                        
                        self._subprocess[subprocess['@type']][subprocess['@id']].append(subevent['@id'])
                        self._subevents[subprocess['@type']][subevent['@id']] = subprocess['@id']
                        
        
    
    # method for fetching activity relation data guards
    def extract_expressions(self):
        """
        method for extracting data-dependent relations, data gaurds and their corresponding link/mapping ID.
        does not provide return value, method manipulates object attributes in-place
        """
        
        expression = self._model.get('dcrgraph').get('specification').get('resources').get('expressions').get('expression')
        # get data for each activity and sanitize format
        for exp in expression:
            exp = list(exp.values())
            e = exp[1].split('=')
            e[0] = e[0].strip()
            e[1] = e[1].strip()
            if (e[0],e[1]) not in self._expressions.keys():
                self._expressions[(e[0],e[1])] = [exp[0]]
            else:
                self._expressions[(e[0],e[1])].append(exp[0])
            
            
    # method for fetching all the activities and assigning booleans for include, pending and extracted in order
    # to keep track of activity state. all states set to false initially
    def extract_activities(self):
        """
        method for extracting activity ID's and append tuple of state.
        does not provide return value, method manipulates object attributes in-place
        """
        
        activity = self._model.get('dcrgraph').get('specification').get('resources').get('labelMappings')
        for key, value in activity.items():
            n = len(value)
            for i in range(0, n):
                self._activities[value[i]['@eventId']] = [False, False, False]
        
    
    # method for fetching initial markings from the dcr-model
    def extract_markings(self):
        """
        method for extracting the initial state of the activities corresponding to the model.
        does not provide return value, method manipulates object attributes in-place
        """
        
        included = self._model.get('dcrgraph').get('runtime').get('marking').get('included')
        pending = self._model.get('dcrgraph').get('runtime').get('marking').get('pendingResponses')
        executed = self._model.get('dcrgraph').get('runtime').get('marking').get('executed')
        
        if included is not None:
            events = included.get('event')
            for event in events:
                self._included.append(event['@id'])
        
        if pending is not None:
            events = pending.get('event')
            for event in events:
                self._pending.append(event['@id'])
        
        if executed is not None:
            events = executed.get('event')
            for event in events:
                self._executed.append(event['@id'])
     
            
    def extract_all(self):
        """
        performs all above extracting methods in sequence.
        does not provide return value, method manipulates object attributes in-place
        """
        
        self.extract_relations()
        self.extract_nested_activities()
        self.extract_activities()
        self.extract_expressions()
        self.extract_markings()

# Class for Conformance Checking
This conformance checking algorithm is made with inspiration from the idea of rule-based checking, trace replay and some of the DCR Debois DCR Miner algorithm. The core idea is; by following the trace, we catch violations by (1) keeping track of in- and outgoing constraints, (2) recording state of events and (3) managing data input. The rules which are defined in the relation of events are what decides if a trace is conform or not.


In [ ]:
class Conformance:
    """ 
    A class used to perform check conformance between a DCR model and DCR log.
    Alogrithm dea is based on rule-based conformance checking and trace replay. 
    Conformity of a trace is checked by replaying a trace on a model while updating model state based on
    active constraints.

    ...

    Attributes
    ----------
   _trace_events : dict
       dictionary containing traces from a Trace class object

    _model_relations : dict
        dictionary containing relations from a ProcessMining class object

    _model_activities : dict
        dictionary containing activities from a ProcessMining class object

    _model_time : dict
        dictionary containing time dependent relations from a ProcessMining class object

    _model_expressions : dict
        dictionary containing data gaurd information from a ProcessMining class object

    _model_expression_relations : dict
        dictionary containing data dependent relations and mappings from a ProcessMining class object

    _model_subevents : dict
        dictionary containing events nested in a subprocess/nesting from a ProcessMining class object

    _model_subprocess : dict
        dictionary containing subprocesses/nestings with corresponding nested events, 
        from a ProcessMining class object

    _model_included : list
        dictionary containing initial included events from a ProcessMining class object

    _model_pending : list
        dictionary containing initial pending events from a ProcessMining class object

    _model_executed : list
        dictionary containing initial excluded events from a ProcessMining class object

    _isExecuted : OrderedSet
        set containing executed events, added to set in order

    _isIncluded : OrderedSet
        set containing included events, added to set in order

    _isPending : OrderedSet
        set containing pending events, added to set in order

    _isExcluded : set
        set containing excluded events

    _violations : dict
        dictionary containing violated activities and the violated constraint

    _isConform : bool
        boolean indicating if the replayed trace is conform or not
            True = Conform Trace
            False = Violating Trace

    Methods
    -------
    rule_checking(event, _id)
        method for handling data-dependent relations and checking their corresponding constraint rule

    check_expressions(trace)
        method for checking if a data-dependant activity has been activated and should be included

    check_include_rule(trace, relation)
        method for handling the include constraint

    check_exclude_rule(trace, relation)
        method for handling the exclude constraint

    check_response_rule(trace, relation)
        method for handling the response constraint

    check_condition_rule(trace, relation)
        method for handling the condition constraint

    trace_replay()
        method for conducting trace replay of log onto model. returns (1) lists representing the 
        state of the model after replay, (2) if log conformity is true or false
    """
    
    def __init__(self, trace, pm): 
        """
        Parameters
        ----------
        trace : Trace object (OrderedDict)
            dictionary containing a trace from a DCR model in from of a Trace class object
        
        pm : ProcessMining object (OrderedDict)
            dictionary containing a DCR model in form of a ProcessMining class object
        """
        
        # fetching trace data
        self._trace_events = trace._events
        
        # fetching model data
        self._model_relations = pm._relations
        self._model_activities = pm._activities
        self._model_time = pm._time
        self._model_expressions = pm._expressions
        self._model_expression_relations = pm._expression_relations
        self._model_subevents = pm._subevents
        self._model_subprocess = pm._subprocess
        self._model_included = pm._included
        self._model_pending = pm._pending
        self._model_executed = pm._executed
        
        #initializing variables
        self._isExecuted = OrderedSet()
        self._isIncluded = OrderedSet()
        self._isPending = OrderedSet()
        self._isExcluded = set()
        
        self._violations = {}
        self._isConform = False 
        
        
    def rule_checking(self, trace_event, _id):
        """
        method for handling data-dependent relations and checking their corresponding constraint rule.
        does not provide return value, method manipulates object attributes in-place
        
        Parameters
        ----------
        trace_event : Trace object (OrderedDict)
            the current data-dependent activity which is executed and evaluated
        
        _id : list
            list of mapping ID's which represents a relation between two activities and its constraint
            
        """
        for mapFromRule in _id:
            for mapToRule, relations in self._model_expression_relations.items():
                if mapFromRule == mapToRule:
                    for relation, constraint in relations.items():
                        
                        
                        self._isExecuted.add(relation[0])
                        self._model_activities[relation[0]][2] = True
                        
                        # handling include rule    
                        if (relation[0] in self._isIncluded and 
                            'include' == constraint and trace_event['@id'] in self._isExecuted):
                            self._isIncluded.add(relation[1])
                            self._model_activities[relation[1]][0] = True
                            self._isExcluded.discard(relation[1])
                        
                        # handling exclude rule
                        if (relation[0] in self._isIncluded and 'exclude' == constraint and 
                            trace_event['@id'] in self._isExecuted):
                            self._model_activities[relation[1]][0] = False
                            self._model_activities[relation[1]][1] = False
                            self._isIncluded.discard(relation[1])
                            self._isPending.discard(relation[1])
                            self._isExcluded.add(relation[1])
                        
                            
                        # handling response rule    
                        if (relation[0] in self._isExecuted and 
                            relation[1] in self._isIncluded and 'response' == constraint):
                            
                            self._model_activities[relation[1]][1] = True
                            self._isPending.add(relation[1])
                        
                        # handling condition rule
                        if (relation[1] in self._isIncluded and 'condition' == constraint):

                            try:
                                idx = list(self._isExecuted).index(trace_event['@id'])
                                if relation[0] not in self._isExecuted[:idx]:
                                    self._violations[trace_event['@id']] = constraint
                            except:
                                pass
                        
                        # add activity relation to relation dictionary
                        if relation not in self._model_relations.keys():
                            self._model_relations[relation] = [constraint]
                        else:
                            self._model_relations[relation].append(constraint)
       
    
    def check_expressions(self, trace_event):
        """
        method for checking if a data-dependant activity has been activated and should be included.
        does not provide return value, method manipulates object attributes in-place
        
        Parameters
        ----------
        trace_event : Trace object (OrderedDict)
            dictionary containing information regarding the current event
        
        """
        
        events = self._model_expressions.items()
        for event in events: 
            if trace_event['@id'] == event[0][0]:
                if trace_event['@data'] == event[0][1]:
                    _id = event[1]
                    _id.sort()
                    self.rule_checking(trace_event,_id)
                    
            if trace_event['@id'] == event[0][1]:
                data = trace_event['@data']
                for data_source in self._trace_events:
                    if data_source['@id'] == event[0][0] and data == data_source['@data']:
                        _id = event[1]
                        _id.sort()
                        self.rule_checking(trace_event,_id)

    
    
    def check_include_rule(self, trace_event, relation):
        """
        method for handling the DCR inclusion constraint.
        a include b
        if activity a has been executed and has an include relation to b, b gets included.
        does not provide return value, method manipulates object attributes in-place
        
        Parameters
        ----------
        trace_event : Trace object (OrderedDict)
            dictionary containing information regarding the current event
        
        relation : tuple
            tuple of the relation related to the current evaluated trace_event
        """
        
        # Handling include rule
        if (trace_event == relation[0] and trace_event in self._isExecuted):
            self._isIncluded.add(relation[1])
            self._model_activities[relation[1]][0] = True
            self._isExcluded.discard(relation[1])
            

            
    def check_response_rule(self, trace_event, relation):
        """
        method for handling the DCR response constraint.
        a response b
        if activity a has been executed and has a response relation to b, 
        b is pending a response iff b is included.
        does not provide return value, method manipulates object attributes in-place
        
        Parameters
        ----------
        trace_event : Trace object (OrderedDict)
            dictionary containing information regarding the current event
        
        relation : tuple
            tuple of the relation related to the current evaluated trace_event
        """
        
        # Handling response rule
        if (trace_event == relation[0] and trace_event in self._isExecuted and relation[1] in self._isIncluded):

            self._model_activities[relation[1]][1] = True
            self._isPending.add(relation[1])

            
    def check_condition_rule(self, trace_event, relation):
        """
        method for handling the DCR condition constraint.
        a condition b
        if activity b has been executed and has an ingoing condition relation from a, 
        a must have been included and executed prior to b.
        does not provide return value, method manipulates object attributes in-place
        
        Parameters
        ----------
        trace_event : Trace object (OrderedDict)
            dictionary containing information regarding the current event
        
        relation : tuple
            tuple of the relation related to the current evaluated trace_event
        """
        # Handling condition rule
        if (trace_event == relation[1]):
            try:
                idx = list(self._isExecuted).index(trace_event)
                if relation[0] not in self._isExecuted[:idx]:
                    self._violations[trace_event] = 'ingoing condition violated'
            except:
                pass
            

    def check_exclude_rule(self, trace_event, relation):
        """
        method for handling the DCR condition constraint.
        a exclude b
        if activity a has been executed and has an exclude relation to b, 
        b is excluded.
        does not provide return value, method manipulates object attributes in-place
        
        Parameters
        ----------
        trace_event : Trace object (OrderedDict)
            dictionary containing information regarding the current event
        
        relation : tuple
            tuple of the relation related to the current evaluated trace_event
        """
        
        # Handling exclude rule
        if (trace_event == relation[0] and trace_event in self._isExecuted):
            self._model_activities[relation[1]][0] = False
            self._model_activities[relation[1]][1] = False
            self._isIncluded.discard(relation[1])
            self._isExcluded.add(relation[1])
            self._isPending.discard(relation[1])        
    
    def trace_replay(self):
        """
        method for conducting trace replay of a DCR trace onto the desired DCR model.
        
        Attributes
        ----------
        nested_events : set
            set comprised of all nested events
        
        nesting_IDs : set
            set comprised of all subprocess/nestings
        
        Returns
        -------
        isIncluded : list
            list of all included activities after a trace replay
            
        isPending : list
            list of all pending activities after a trace replay
            
        isExecuted : list
            list of all executed activities after a trace replay
            
        isExcluded : list
            list of all excluded activities after a trace replay
            
        isConform : bool
            boolean indicating wheter the trace is conform or not
                True = Conform Trace
                False = Violating Trace
        
        """
        
        nested_events = set()
        nesting_IDs = set()
        
        # extract all nested event ID's and nesting ID's
        temp_nested_events = list(self._model_subevents.values())
        for events in temp_nested_events:
            for event in events.values():
                nesting_IDs.add(event)
            for events in events.keys():
                nested_events.add(event)
        
        # iterate through trace_events
        for trace_event in self._trace_events:
            

            # getting all included activities which are subprocesses/nesting etc
            for pending_event in self._isPending:
                if pending_event in nesting_IDs:
                    for subevent in list(self._model_subevents.values()):
                        for key, value in subevent.items():
                            if pending_event == value and pending_event in self._isIncluded:
                                if key in self._model_included or key in self._isIncluded: 
                                    self._isIncluded.add(key)
                                    self._model_activities[key][0] = True
                                else:
                                    self._isExcluded.add(key)

            # find all included activities in the model which are not nested and add to inclusion list
            for included_event in self._model_included:
                if trace_event['@id'] == included_event and trace_event['@id'] not in nested_events:

                    self._model_activities[trace_event['@id']][0] = True
                    self._isIncluded.add(trace_event['@id'])
            
            # add trace_event to violating if excluded
            if trace_event['@id'] in self._isExcluded:
                self._violations[trace_event['@id']] = 'excluded activity executed'

            # add trace_event to list of executions if it exists in the model
            for model_event in self._model_activities:
                if trace_event['@id'] == model_event:
                    self._model_activities[trace_event['@id']][2] = True
                    self._model_activities[trace_event['@id']][1] = False
                    self._isExecuted.add(trace_event['@id'])
            
            
            if trace_event['@id'] in self._isIncluded:

            # check relation activity for current trace_event
                for relation in self._model_relations.items():
                    
                    # if relation exists
                    if trace_event['@id'] == relation[0][0]:

                        if 'include' in relation[1]:
                            self.check_include_rule(trace_event['@id'], relation[0])
                        if 'exclude' in relation[1]:
                            self.check_exclude_rule(trace_event['@id'], relation[0])
                        if 'response' in relation[1]:    
                            self.check_response_rule(trace_event['@id'], relation[0])
                        if 'condition' in relation[1]:
                            self.check_condition_rule(trace_event['@id'], relation[0])
                
                # handle data-dependent activity
                if '@data' in trace_event.keys():
                    self.check_expressions(trace_event)



            # replacing pending and included subprocesses with it's nested ID's
            temp_pending = list(self._isPending)
            for pending_event in temp_pending:
                cnt = 0

                for nesting_type, events in self._model_subevents.items(): 
                    for subevent, subprocess in events.items():

                        # handling nesting type subprocess and adding subevents if the whole process 
                        # has been included
                        if nesting_type == 'nesting' and subprocess == pending_event:
                            self._isPending.add(subevent)
                            self._isIncluded.add(subevent)
                            self._model_activities[subevent][0] = True
                            self._model_activities[subevent][1] = True
                        if nesting_type == 'subprocess' and trace_event['@id'] == subevent:
                            self._isPending.add(subprocess)

                        # counting activities executed in a specific subprocess/nesting
                        if (subprocess == pending_event) and subevent in self._isExecuted:
                            cnt = 1 + cnt

                    for process_type, events in self._model_subprocess.items():
                        for subprocess, subevents in events.items():
                            
                            # updating the state of the whole subprocess if all subprocess events have been
                            # executed and is not pending
                            if pending_event == subprocess:
                                if cnt == len(subevents):

                                    for relation in self._model_relations.items():

                                        if 'include' in relation[1]:
                                            self.check_include_rule(pending_event, relation[0])
                                        if 'exclude' in relation[1]:
                                            self.check_exclude_rule(pending_event, relation[0])
                                        if 'response' in relation[1]:    
                                            self.check_response_rule(pending_event, relation[0])
                                        if 'condition' in relation[1]:
                                            self.check_condition_rule(pending_event, relation[0])

                                    self._isExecuted.add(pending_event)
                                    self._model_activities[pending_event][2] = True

        
        # removing events from pending if they have been executed, else put them in violation list
        temp_pending2 = list(self._isPending)
        for pending_event in temp_pending2:
            if pending_event in self._isExecuted:
                if pending_event in self._isExcluded:
                    self._violations[pending_event] = 'executed but excluded'
                else:
                    self._isPending.discard(pending_event)
                    self._model_activities[pending_event][1] = False
            
            else:
                self._violations[pending_event] = 'is pending'
                
        # requirement for any trace to be conform is not having violations and no pending events
        if set(self._isIncluded) == (set(self._isExecuted)-set(self._isExcluded)) and (
            len(self._isPending) == 0) and len(self._violations) == 0:
            self._isConform = True
        
        return self._isConform, set(self._isIncluded), set(self._isPending), set(self._isExecuted), set(self._isExcluded)
    
    
   


# Loading the traces

Insert the path to the log as a variable or insert a folder for multiple traces in order to produce a list of logs. Iterating in a folder only gathers XML logs.

In [ ]:
PATH = 'example/directory/file.xml'

violations = []
valid = []

# iterating though folder in fetch logs
for filename in glob.glob(os.path.join('data/violating/','*.xml')): 
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        violations.append(xmltodict.parse(f.read()))
        
for filename in glob.glob(os.path.join('data/valid/','*.xml')):
    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        valid.append(xmltodict.parse(f.read()))        
    

# Loading the models
Insert a model path in the same fashion as below code.

In [ ]:
# model XML log path

model_path = 'data/IR.xml'


# MODEL

fd = open(model_path)
xmlstr = fd.read()
model = xmltodict.parse(xmlstr)


# Performing the Conformance Checking

In [ ]:
for idx, log in enumerate(valid):
    trace = Trace(log)
    trace.extract_events()
    
    pm = ProcessModel(model)
    pm.extract_all()

    cc = Conformance(trace, pm)
    cc.trace_replay()
    if cc._isConform:
        print("Trace {} is conform: {}".format(idx+1, cc._isConform))
    else:
        print("Trace {} is violating: {}".format(idx+1, cc._violations))


In [ ]:
for idx, log in enumerate(violations):
    trace = Trace(log)
    trace.extract_events()
    
    pm = ProcessModel(model)
    pm.extract_all()

    cc = Conformance(trace, pm)
    cc.trace_replay()
    if cc._isConform:
        print("Trace {} is conform: \n{}\n".format(idx, cc._isConform))
    else:
        print("Trace {} is violating: \n{}\n".format(idx, cc._violations))